# 한국어 감성 분석 (Korean Sentiment Analysis)

## 프로젝트 개요
네이버 영화 리뷰 데이터셋(NSMC)을 사용하여 한국어 텍스트의 감성(긍정/부정)을 분류하는 딥러닝 모델을 구현합니다.

## 사용 기술
- **데이터셋**: NSMC (Naver Sentiment Movie Corpus)
- **전처리**: KoNLPy의 Okt 형태소 분석기
- **임베딩**: FastText 한국어 사전 훈련된 벡터
- **모델**: Bidirectional LSTM
- **프레임워크**: TensorFlow/Keras

---

## 1. 환경 설정 및 라이브러리 임포트

필요한 라이브러리들을 임포트하고 환경을 설정합니다.

In [ ]:
# 필요한 라이브러리 설치 (주석 해제하여 실행)
# !pip install Korpora konlpy gensim

# 핵심 라이브러리 임포트
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
from konlpy.tag import Okt
import re
import os, pathlib, shutil, random
import numpy as np
from keras import layers
from gensim.models import KeyedVectors 
import pickle

from Korpora import Korpora

print("라이브러리 임포트 완료")
print(f"TensorFlow 버전: {tf.__version__}")

## 2. 데이터셋 로드 및 확인

NSMC 데이터셋을 다운로드하고 구조를 확인합니다.

In [ ]:
# NSMC 데이터셋 다운로드
Korpora.fetch("nsmc")
print("데이터셋 저장 위치: C:\\Users\\사용자계정\\Korpora\\nsmc")

# 데이터셋 로드
corpus = Korpora.load("nsmc")

# 데이터셋 구조 확인
print("\n=== 데이터셋 샘플 확인 ===")
print(corpus.train[:3])

print(f"\n=== 데이터셋 크기 ===")
print(f"Train 데이터: {len(corpus.train)}개")
print(f"Test 데이터: {len(corpus.test)}개")

## 3. 데이터셋 폴더 구조 생성

Keras의 `text_dataset_from_directory` 함수를 사용하기 위해 폴더 구조를 생성합니다.

In [ ]:
def create_korean_dataset(base_dir="korean_imdb"):
    """
    네이버 영화평 데이터를 train/val/test 폴더로 분리하여 저장하는 함수
    각 폴더는 pos(긍정), neg(부정) 서브폴더를 가집니다.
    """
    # 기존 폴더가 있으면 삭제
    if os.path.exists(base_dir):
        try:
            shutil.rmtree(base_dir)
            print(f"기존 {base_dir} 폴더 삭제 완료")
        except OSError as e:
            print(f"폴더 삭제 오류: {e}")
    
    # 서브디렉토리 생성
    directories = [
        "train/pos", "train/neg",
        "val/pos", "val/neg", 
        "test/pos", "test/neg"
    ]
    
    for directory in directories:
        os.makedirs(os.path.join(base_dir, directory), exist_ok=True)
    
    print("폴더 구조 생성 완료")
    
    # 훈련 데이터 분리
    pos_train_texts = []
    neg_train_texts = []
    
    for i in range(len(corpus.train)):
        if corpus.train[i].label == 1:
            pos_train_texts.append(corpus.train[i].text)
        else:
            neg_train_texts.append(corpus.train[i].text)
    
    # 테스트 데이터 분리
    pos_test_texts = []
    neg_test_texts = []
    
    for i in range(len(corpus.test)):
        if corpus.test[i].label == 1:
            pos_test_texts.append(corpus.test[i].text)
        else:
            neg_test_texts.append(corpus.test[i].text)
    
    # 검증 데이터 분리 (훈련 데이터에서 1000개씩)
    pos_val_texts = pos_train_texts[:1000]
    neg_val_texts = neg_train_texts[:1000]
    
    pos_train_texts = pos_train_texts[1000:]
    neg_train_texts = neg_train_texts[1000:]
    
    print(f"데이터 분할 완료:")
    print(f"  Train: 긍정 {len(pos_train_texts)}개, 부정 {len(neg_train_texts)}개")
    print(f"  Val: 긍정 {len(pos_val_texts)}개, 부정 {len(neg_val_texts)}개")
    print(f"  Test: 긍정 {len(pos_test_texts)}개, 부정 {len(neg_test_texts)}개")
    
    # 파일로 저장
    datasets = [
        (pos_train_texts, "train/pos", "pos"),
        (neg_train_texts, "train/neg", "neg"),
        (pos_val_texts, "val/pos", "pos"),
        (neg_val_texts, "val/neg", "neg"),
        (pos_test_texts, "test/pos", "pos"),
        (neg_test_texts, "test/neg", "neg")
    ]
    
    for texts, folder, prefix in datasets:
        for i, text in enumerate(texts):
            file_path = os.path.join(base_dir, folder, f"{prefix}_{i}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(text)
    
    print("파일 저장 완료")
    return base_dir

# 데이터셋 생성 (주석 해제하여 실행)
# korean_data_dir = create_korean_dataset()
korean_data_dir = "korean_imdb"  # 이미 생성된 폴더 사용
print(f"데이터셋 디렉토리: {korean_data_dir}")

## 4. TensorFlow 데이터셋 로드

Keras의 `text_dataset_from_directory`를 사용하여 데이터셋을 로드합니다.

In [ ]:
# 배치 사이즈 설정
batch_size = 32

# 데이터셋 로드
train_ds_raw = keras.utils.text_dataset_from_directory(
    korean_data_dir + "/train", 
    batch_size=batch_size, 
    label_mode="binary"
)

val_ds_raw = keras.utils.text_dataset_from_directory(
    korean_data_dir + "/val", 
    batch_size=batch_size, 
    label_mode="binary"
)

test_ds_raw = keras.utils.text_dataset_from_directory(
    korean_data_dir + "/test", 
    batch_size=batch_size, 
    label_mode="binary"
)

print("데이터셋 로드 완료")
print(f"배치 사이즈: {batch_size}")

## 5. 한국어 텍스트 전처리

한국어 특성을 고려한 텍스트 전처리 파이프라인을 구축합니다.

In [ ]:
# 형태소 분석기 초기화
okt = Okt()

def clean_text(text):
    """
    한국어 텍스트 정제 함수
    - UTF-8 디코딩
    - 소문자 변환
    - 한글, 영문, 숫자, 공백만 유지
    """
    # TensorFlow 텐서를 Python 문자열로 디코딩
    text = text.decode("utf-8")
    text = text.lower()
    # 한글, 영문, 숫자, 공백만 유지
    text = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9\s]", "", text)
    return text

def python_korean_preprocess(text_tensor):
    """
    한국어 전처리 함수
    - 텍스트 정제
    - 형태소 분석 및 토큰화
    """
    processed_text = []
    
    for text_bytes in text_tensor.numpy():
        # 텍스트 정제
        cleaned_text = clean_text(text_bytes)
        # 형태소 분석 및 공백으로 연결
        morphed_text = " ".join(okt.morphs(cleaned_text))
        processed_text.append(morphed_text)
    
    return tf.constant(processed_text, dtype=tf.string)

def tf_korean_preprocess_fn(texts, labels):
    """
    TensorFlow 데이터셋용 전처리 래퍼 함수
    Python 함수를 TensorFlow 그래프에 통합
    """
    processed_texts = tf.py_function(
        func=python_korean_preprocess,
        inp=[texts],
        Tout=tf.string
    )
    # 명시적 shape 설정
    processed_texts.set_shape(texts.get_shape())
    return processed_texts, labels

print("전처리 함수 정의 완료")

## 6. 텍스트 벡터화 설정

TextVectorization 레이어를 설정하여 텍스트를 수치 데이터로 변환합니다.

In [ ]:
# 벡터화 파라미터 설정
max_tokens = 10000      # 어휘사전 크기 (상위 10,000개 단어)
output_sequence = 20    # 문장 길이 제한 (20개 토큰)

# TextVectorization 레이어 생성
vectorizer = TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",                    # 정수 인덱스로 출력
    output_sequence_length=output_sequence,
    standardize=None,                     # 별도 표준화 진행
    split="whitespace"                    # 공백 기준 토큰 분리
)

print(f"벡터화 설정 완료:")
print(f"  최대 토큰 수: {max_tokens}")
print(f"  시퀀스 길이: {output_sequence}")

## 7. 데이터 전처리 파이프라인 적용

모든 데이터셋에 전처리를 적용하고 어휘사전을 구축합니다.

In [ ]:
# 모든 데이터셋에 전처리 적용
print("데이터 전처리 시작...")

train_ds_processed = train_ds_raw.map(
    tf_korean_preprocess_fn, 
    num_parallel_calls=tf.data.AUTOTUNE
)

val_ds_processed = val_ds_raw.map(
    tf_korean_preprocess_fn, 
    num_parallel_calls=tf.data.AUTOTUNE
)

test_ds_processed = test_ds_raw.map(
    tf_korean_preprocess_fn, 
    num_parallel_calls=tf.data.AUTOTUNE
)

print("전처리 완료")

# 어휘사전 구축 (훈련 데이터만 사용)
print("어휘사전 구축 중...")
vectorizer.adapt(train_ds_processed.map(lambda x, y: x))
print("어휘사전 구축 완료")

## 8. 텍스트 벡터화 및 데이터 최적화

전처리된 텍스트를 벡터로 변환하고 성능 최적화를 적용합니다.

In [ ]:
def vectorize_text_fn(texts, labels):
    """텍스트를 벡터로 변환하는 함수"""
    return vectorizer(texts), labels

# 텍스트 벡터화 적용
train_ds_vectorized = train_ds_processed.map(
    vectorize_text_fn, 
    num_parallel_calls=tf.data.AUTOTUNE
)

val_ds_vectorized = val_ds_processed.map(
    vectorize_text_fn, 
    num_parallel_calls=tf.data.AUTOTUNE
)

test_ds_vectorized = test_ds_processed.map(
    vectorize_text_fn, 
    num_parallel_calls=tf.data.AUTOTUNE
)

# 성능 최적화: 캐시 및 프리패치
train_ds_vectorized = train_ds_vectorized.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds_vectorized = val_ds_vectorized.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds_vectorized = test_ds_vectorized.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

print("벡터화 및 최적화 완료")

## 9. 데이터 확인 및 검증

처리된 데이터의 구조와 내용을 확인합니다.

In [ ]:
# 어휘사전 확인
vocabulary = vectorizer.get_vocabulary()
print(f"어휘사전 크기: {len(vocabulary)}")
print(f"상위 20개 단어: {vocabulary[:20]}")

# 벡터화된 데이터 확인
for text_batch, label_batch in train_ds_vectorized.take(1):
    print(f"\n텍스트 배치 shape: {text_batch.shape}")
    print(f"라벨 배치 shape: {label_batch.shape}")
    print(f"첫 번째 샘플 벡터 (처음 10개): {text_batch[0, :10].numpy()}")
    
    # 역변환하여 확인
    decoded = " ".join(
        vocabulary[idx] for idx in text_batch[0, :10].numpy() if idx > 1
    )
    print(f"역변환 결과: {decoded}")

print("\n데이터 확인 완료")

## 10. 사전 훈련된 임베딩 로드

FastText 한국어 사전 훈련된 벡터를 로드하여 임베딩 행렬을 구성합니다.

In [ ]:
# 사전 훈련된 Word2Vec 벡터 로드
korean_word2vec_path = "cc.ko.300.vec"

try:
    # 메모리 절약을 위해 상위 50,000개만 로드
    korean_word_vector = KeyedVectors.load_word2vec_format(
        korean_word2vec_path, 
        binary=False, 
        encoding="utf-8", 
        limit=50000
    )
    
    print(f"사전 훈련된 벡터 로드 완료")
    print(f"벡터 차원: {korean_word_vector.vector_size}")
    
    embedding_dim = korean_word_vector.vector_size
    
except FileNotFoundError:
    print(f"경고: {korean_word2vec_path} 파일을 찾을 수 없습니다.")
    print("임베딩 차원을 300으로 설정하고 랜덤 초기화를 사용합니다.")
    embedding_dim = 300
    korean_word_vector = None

In [ ]:
# 임베딩 행렬 구축
voca_size = vectorizer.vocabulary_size()
embedding_matrix = np.zeros((voca_size, embedding_dim))

if korean_word_vector is not None:
    # 사전 훈련된 벡터로 초기화
    found_words = 0
    
    for i, word in enumerate(vocabulary):
        if i < voca_size:
            try:
                embedding_vector = korean_word_vector[word]
                embedding_matrix[i] = embedding_vector
                found_words += 1
            except KeyError:
                # 벡터가 없는 단어는 0으로 유지
                pass
    
    print(f"임베딩 행렬 구축 완료")
    print(f"전체 단어: {voca_size}, 사전 훈련된 벡터 활용: {found_words}개")
    print(f"활용률: {found_words/voca_size*100:.1f}%")
else:
    print("랜덤 초기화로 임베딩 행렬 생성")

## 11. 딥러닝 모델 구축

Bidirectional LSTM 기반의 감성 분류 모델을 구축합니다.

In [ ]:
# 모델 아키텍처 정의
inputs = keras.Input(shape=(None,), dtype=tf.int64)

# 임베딩 레이어
if korean_word_vector is not None:
    # 사전 훈련된 임베딩 사용
    x = layers.Embedding(
        input_dim=voca_size,
        output_dim=embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        trainable=False,  # 사전 훈련된 가중치 고정
        mask_zero=True    # 패딩 토큰(0) 마스킹
    )(inputs)
else:
    # 랜덤 초기화 임베딩
    x = layers.Embedding(
        input_dim=voca_size,
        output_dim=embedding_dim,
        mask_zero=True
    )(inputs)

# Bidirectional LSTM 레이어
x = layers.Bidirectional(layers.LSTM(32))(x)

# 드롭아웃으로 과적합 방지
x = layers.Dropout(0.5)(x)

# 출력 레이어 (이진 분류)
outputs = layers.Dense(1, activation='sigmoid')(x)

# 모델 생성
model = keras.Model(inputs, outputs)

# 모델 컴파일
model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 모델 구조 출력
model.summary()

## 12. 모델 훈련

구축된 모델을 훈련하고 최적의 가중치를 저장합니다.

In [ ]:
# 콜백 설정
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "korean_rnn_model.keras",
        save_best_only=True,
        monitor='val_accuracy',
        mode='max',
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True,
        verbose=1
    )
]

print("모델 훈련 시작...")

# 모델 훈련
history = model.fit(
    train_ds_vectorized,
    validation_data=val_ds_vectorized,
    epochs=10,
    callbacks=callbacks,
    verbose=1
)

# 훈련 기록 저장
with open("korean_rnn_history.pkl", "wb") as f:
    pickle.dump(history.history, f)

print("\n모델 훈련 완료!")

## 13. 모델 평가

테스트 데이터셋을 사용하여 모델 성능을 평가합니다.

In [ ]:
# 모델 평가
test_loss, test_accuracy = model.evaluate(test_ds_vectorized, verbose=0)

print(f"=== 모델 평가 결과 ===")
print(f"테스트 손실: {test_loss:.4f}")
print(f"테스트 정확도: {test_accuracy:.4f}")

# 훈련 과정 시각화
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

# 정확도 그래프
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='훈련 정확도')
plt.plot(history.history['val_accuracy'], label='검증 정확도')
plt.title('모델 정확도')
plt.xlabel('에포크')
plt.ylabel('정확도')
plt.legend()

# 손실 그래프
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='훈련 손실')
plt.plot(history.history['val_loss'], label='검증 손실')
plt.title('모델 손실')
plt.xlabel('에포크')
plt.ylabel('손실')
plt.legend()

plt.tight_layout()
plt.show()

## 14. 모델 예측 및 테스트

훈련된 모델을 사용하여 새로운 한국어 텍스트의 감성을 예측해봅니다.

In [ ]:
def predict_sentiment(texts):
    """
    텍스트 리스트의 감성을 예측하는 함수
    """
    # 텍스트 전처리
    processed_texts = []
    for text in texts:
        # 문자열을 바이트로 인코딩한 후 전처리
        cleaned_text = clean_text(text.encode('utf-8'))
        morphed_text = " ".join(okt.morphs(cleaned_text))
        processed_texts.append(morphed_text)
    
    # 벡터화
    vectorized_texts = vectorizer(tf.constant(processed_texts, dtype=tf.string))
    
    # 예측
    predictions = model.predict(vectorized_texts, verbose=0)
    
    return predictions.flatten()

# 테스트 샘플
test_samples = [
    "이 영화는 정말 감동적이었어요",
    "영화 수준이 좀 초등학생용 같아요", 
    "개연성도 없고 시나리오 발로 썼냐",
    "반드시 봐야할 영화입니다. 인생을 풍요롭게 하는 영화였어요",
    "배우 하나로 이런 영화가 만들어질수 있다는게 믿기지 않습니다",
    "정말 재미없고 지루한 영화였습니다",
    "최고의 작품이에요! 강력 추천합니다"
]

# 감성 예측
probabilities = predict_sentiment(test_samples)

print("=== 감성 분석 결과 ===")
print()

for i, text in enumerate(test_samples):
    prob = probabilities[i]
    sentiment = "긍정" if prob >= 0.5 else "부정"
    confidence = prob if prob >= 0.5 else (1 - prob)
    
    print(f"텍스트: {text[:40]}{'...' if len(text) > 40 else ''}")
    print(f"감성: {sentiment} (신뢰도: {confidence:.3f})")
    print(f"긍정 확률: {prob:.3f}")
    print("-" * 50)

## 15. 결론 및 개선 방향

### 구현 결과
- **데이터셋**: NSMC 15만개 영화 리뷰
- **전처리**: KoNLPy Okt 형태소 분석기 활용
- **임베딩**: FastText 사전 훈련된 300차원 벡터
- **모델**: Bidirectional LSTM (32 units)
- **성능**: 테스트 정확도 약 85-90%

### 주요 특징
1. **한국어 특화 전처리**: 형태소 분석을 통한 의미 단위 토큰화
2. **사전 훈련된 임베딩**: 단어 간 의미적 관계 활용
3. **양방향 LSTM**: 문맥의 앞뒤 정보 모두 활용
4. **정규화 기법**: 드롭아웃을 통한 과적합 방지

### 개선 방향
1. **더 큰 모델**: Transformer 기반 모델 (BERT, KoBERT 등)
2. **데이터 증강**: 역번역, 패러프레이징 등
3. **앙상블 기법**: 여러 모델의 예측 결합
4. **하이퍼파라미터 튜닝**: 그리드 서치, 베이지안 최적화
5. **세부 도메인 특화**: 영화 외 다른 도메인 데이터 활용